In [1]:
import pandas as pd

In [5]:
data_movies = pd.read_csv('movies.csv')

def label_year (row):
    return row['title'][-4:-1]

def title_without_year(row):
    return row['title'][0:-7]

data_movies['year'] = data_movies.apply(lambda row: label_year(row), axis=1)
data_movies['title'] = data_movies.apply(lambda row: title_without_year(row), axis=1)

print('this dataset contains: ', len(data_movies), 'movies')
data_movies.iloc[:10]

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,995
1,2,Jumanji,Adventure|Children|Fantasy,995
2,3,Grumpier Old Men,Comedy|Romance,995
3,4,Waiting to Exhale,Comedy|Drama|Romance,995
4,5,Father of the Bride Part II,Comedy,995
5,6,Heat,Action|Crime|Thriller,995
6,7,Sabrina,Comedy|Romance,995
7,8,Tom and Huck,Adventure|Children,995
8,9,Sudden Death,Action,995
9,10,GoldenEye,Action|Adventure|Thriller,995


In [6]:
data_ratings = pd.read_csv('ratings.csv')
print('this dataset contains: ', len(data_ratings), 'ratings')
data_ratings.iloc[:10]

this dataset contains:  25000095 ratings


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414
